In [1]:
!pip -q install colorlog
!pip -q install keras_radam
!pip -q install keras_tqdm
#!pip install tensorflow==2.1.0-rc1

# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
#os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
os.chdir('..'+sep_local+'..'+sep_local+'..')

print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


In [3]:
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


In [4]:
%cd /content/
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/

[WinError 2] The system cannot find the file specified: '/content/'
C:\Users\Khalid\Documents\projects\Generative_Models\notebooks\losses_evaluation\3shapes
[WinError 3] The system cannot find the path specified: '/content/Generative_Models/'
C:\Users\Khalid\Documents\projects\Generative_Models\notebooks\losses_evaluation\3shapes


Access is denied.


In [5]:
#os.chdir('/content/Generative_Models/')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


# Dataset loading

In [6]:
dataset_name = 'shapes3d'
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

tfds.core.DatasetInfo(
    name='shapes3d',
    version=0.1.0,
    description='3dshapes is a dataset of 3D shapes procedurally generated from 6 ground truth
independent latent factors. These factors are *floor colour*, *wall colour*, *object colour*,
*scale*, *shape* and *orientation*.

All possible combinations of these latents are present exactly once, generating N = 480000 total images.

### Latent factor values

*   floor hue: 10 values linearly spaced in [0, 1]
*   wall hue: 10 values linearly spaced in [0, 1]
*   object hue: 10 values linearly spaced in [0, 1]
*   scale: 8 values linearly spaced in [0, 1]
*   shape: 4 values in [0, 1, 2, 3]
*   orientation: 15 values linearly spaced in [-30, 30]

We varied one latent at a time (starting from orientation, then shape, etc), and sequentially stored the images in fixed order in the `images` array. The corresponding values of the factors are stored in the same order in the `labels` array.
',
    urls=['https://github.com/deepmind/3d-

In [7]:
intermediate_dim = 20
inputs_shape=(64, 64, 3) # image shape
batch_size = 32
latents_dim = 6

In [8]:
TRAIN_BUF = 600
TEST_BUF = 100

In [9]:
import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)

In [10]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/ShapeResults/
MAIN_SAVE_DIR = '/content/drive/My Drive/ShapeResults'

ModuleNotFoundError: No module named 'google.colab'

In [17]:
from data.gt_load.datasets import load

In [18]:
#!mv /content/drive/My\ Drive/3dshapes.h5 /content/Generative_Models/data

In [19]:
#DATA_DOWN_PATH = '..'+sep_local+'..'+sep_local+'..'+sep_local+'data'
#DATA_DOWN_PATH = os.getcwd() + sep_local+'data'
#Script_dir = os.getcwd() + sep_local+'data'+sep_local+'download_gt_data.sh'
# Script call to download "dsprites_full" dataset_name 
#!/bin/bash $Script_dir -f $DATA_DOWN_PATH -d $dataset_name

In [20]:
DATA_PATH =  os.getcwd() + sep_local+'data' +sep_local+'.3dshapes'+sep_local+'3dshapes.h5'
#absolute = abspath(DATA_PATH)
print(DATA_PATH)

C:\Users\Khalid\Documents\projects\Generative_Models\data\.3dshapes\3dshapes.h5


In [21]:
#!ls /content/Generative_Models/data/.gt_datasets/dsprites

In [22]:
#DATA_PATH = r"..\..\..\..\data\.3dshapes\3dshapes.h5"

In [23]:
#import h5py
# load dataset
#dataset = h5py.File(DATA_PATH, 'r')

In [ ]:
eval_dataset = load(dataset_name='shapes3d', dataset_path=DATA_PATH)

In [0]:
next(train_ds)

In [0]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'].numpy().max())
    break

In [0]:
_instance_scale

In [0]:
import numpy as np
from collections.abc import Iterable

In [0]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [0]:
_outputs_shape

4096

# Model's Layers definition

In [0]:
menc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

venc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [0]:
from google.colab import drive
drive.mount('/content/drive')
#!mkdir /content/drive/My\ Drive/DstResults/
MAIN_SAVE_DIR = '/content/drive/My Drive/DstResults'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
model_name = dataset_name+'-sharpdiff'
#experiments_dir='experiments'+sep_local+model_name
recording_dir=MAIN_SAVE_DIR + sep_local  +'recording-Dense'+sep_local + model_name

In [0]:
from training.autoencoding_basic.autoencoders.tVAE import tVAE as AE

In [0]:
from os.path import abspath
absolute = abspath(recording_dir)
print("Recording_dir",absolute)

Recording_dir /content/drive/My Drive/DstResults/recording-Dense/dsprites-sharpdiff


In [0]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    },
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist

In [0]:
_restore = os.path.join(recording_dir, 'var_save_dir')

In [0]:
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)

Restore_dir /content/drive/My Drive/DstResults/recording-Dense/dsprites-sharpdiff/var_save_dir


In [0]:
#to restore trained model, set filepath=_restore

In [0]:
ae = AE( 
    name=model_name,
    df=1,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=_restore
    )

  DEBUG    | Restore old models ...


  WARNING  | None


Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 64, 64, 1)]       0         
_________________________________________________________________
dense (Dense)                (None, 64, 64, 10)        20        
_________________________________________________________________
dense_1 (Dense)              (None, 64, 64, 10)        110       
_________________________________________________________________
flatten (Flatten)            (None, 40960)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 245766    
_________________________________________________________________
batch_normalization (BatchNo (None, 6)                 24        
_________________________________________________________________
dropout (Dropout)            (None, 6)              

  WARNING  | None


Model: "inference_logvariance"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_logvariance_inputs [(None, 64, 64, 1)]       0         
_________________________________________________________________
dense_3 (Dense)              (None, 64, 64, 10)        20        
_________________________________________________________________
dense_4 (Dense)              (None, 64, 64, 10)        110       
_________________________________________________________________
flatten_1 (Flatten)          (None, 40960)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 245766    
_________________________________________________________________
batch_normalization_1 (Batch (None, 6)                 24        
_________________________________________________________________
dropout_1 (Dropout)          (None, 6)       

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 6)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_7 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_8 (Dense)              (None, 4096)              86016     
_________________________________________________________________
reshape (Reshape)            (None, 64, 64, 1)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 1)         4         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 1)         0

In [0]:
from evaluation.quantitive_metrics.sharp_difference import prepare_sharpdiff

In [0]:
from statistical.losses_utilities import similarity_to_distance

In [0]:
ae.compile(loss={'x_logits': similarity_to_distance(prepare_sharpdiff([ae.batch_size]+ae.get_inputs_shape()))})

Model: "dsprites-sharpdiff"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (I [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
inference_logvariance (Model)   (None, 6)            245920      inference_logvariance_inputs[0][0
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 6)]          0           inference_logvariance[1][0]      
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 6)]          0           tf_op_layer_Mul[0][0]            
_________________________________________________________________________________

# Callbacks

In [0]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver
from training.callbacks.disentangle_supervied import DisentanglementSuperviedMetrics
from training.callbacks.disentangle_unsupervied import DisentanglementUnsuperviedMetrics

In [0]:
gts_csv = os.path.join(recording_dir, 'csv_dir', 'gts_metrics')
gtu_csv = os.path.join(recording_dir, 'csv_dir', 'gtu_metrics')

In [0]:
gts_mertics = DisentanglementSuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gts_csv,
    num_train=10000,
    num_test=200,
    batch_size=batch_size,
    continuous_factors=False,
    gt_freq=10
)
gtu_mertics = DisentanglementUnsuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gtu_csv,
    num_train=10000,
    num_test=200,
    batch_size=batch_size,
    gt_freq=10
)

In [0]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [0]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [0]:
csv_dir = os.path.join(recording_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'/content/drive/My Drive/DstResults/recording-Dense/dsprites-sharpdiff/csv_dir/dsprites-sharpdiff.csv'

In [0]:
image_gen_dir = os.path.join(recording_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [0]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

# Model Training

In [0]:
ae.fit(
    x=train_ds,
    input_kw='image',
    steps_per_epoch=1000,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg, gts_mertics, gtu_mertics],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1000
)

Train for 1000 steps, validate for 1000 steps
Epoch 1/1000000


/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `nump

1000/1000 - 2116s - loss: -7.4401e+00 - x_logpdf_loss: -1.9248e+00 - x_logits_loss: -6.9336e+00 - z_latents_loss: 1.4168 - x_logits_psnr: 6.0256 - x_logits_total_variation: 348.5990 - x_logits_ssim: 0.7417 - x_logits_sharpdiff: 14.8673 - x_logits_mean_absolute_error: 0.4994 - x_logits_mean_squared_error: 0.2497 - val_loss: -7.0186e+00 - val_x_logpdf_loss: -1.7085e+00 - val_x_logits_loss: -7.0396e+00 - val_z_latents_loss: 1.7290 - val_x_logits_psnr: 6.0415 - val_x_logits_total_variation: 81.6355 - val_x_logits_ssim: 0.7207 - val_x_logits_sharpdiff: 15.0791 - val_x_logits_mean_absolute_error: 0.4986 - val_x_logits_mean_squared_error: 0.2488
Epoch 2/1000000
1000/1000 - 1458s - loss: -7.5301e+00 - x_logpdf_loss: -1.9135e+00 - x_logits_loss: -7.0059e+00 - z_latents_loss: 1.3889 - x_logits_psnr: 6.0244 - x_logits_total_variation: 141.8037 - x_logits_ssim: 0.8204 - x_logits_sharpdiff: 15.0117 - x_logits_mean_absolute_error: 0.4996 - x_logits_mean_squared_error: 0.2498 - val_loss: -7.1207e+00 

/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `nump

1000/1000 - 1885s - loss: -7.6403e+00 - x_logpdf_loss: -1.7901e+00 - x_logits_loss: -7.0470e+00 - z_latents_loss: 1.1968 - x_logits_psnr: 6.0207 - x_logits_total_variation: 4.6266 - x_logits_ssim: 0.8442 - x_logits_sharpdiff: 15.0941 - x_logits_mean_absolute_error: 0.5000 - x_logits_mean_squared_error: 0.2500 - val_loss: -7.6527e+00 - val_x_logpdf_loss: -1.7899e+00 - val_x_logits_loss: -7.0471e+00 - val_z_latents_loss: 1.1843 - val_x_logits_psnr: 6.0209 - val_x_logits_total_variation: 0.3518 - val_x_logits_ssim: 0.8440 - val_x_logits_sharpdiff: 15.0942 - val_x_logits_mean_absolute_error: 0.5000 - val_x_logits_mean_squared_error: 0.2500
Epoch 12/1000000
1000/1000 - 1440s - loss: -7.6396e+00 - x_logpdf_loss: -1.7898e+00 - x_logits_loss: -7.0464e+00 - z_latents_loss: 1.1967 - x_logits_psnr: 6.0207 - x_logits_total_variation: 4.5657 - x_logits_ssim: 0.8442 - x_logits_sharpdiff: 15.0929 - x_logits_mean_absolute_error: 0.5000 - x_logits_mean_squared_error: 0.2500 - val_loss: -7.6525e+00 - va

/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `nump

# Model Evaluation

## inception_score

In [0]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [0]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean.compute()}, sigma: {is_sigma.compute()}')

calculating the inception_score mean ...


0it [00:00, ?it/s]

calculating the inception_score sigma ...


/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:780: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
1it [00:37, 37.74s/it]

inception_score mean: [0.9930527], sigma: 2.118794304111987e-11


In [0]:
is_mean.compute()

array([0.99196445])

In [0]:
is_sigma.compute()

5.4622730140814627e-11

## Frechet_inception_distance

In [0]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [0]:
def data_generator():
  for data in train_ds: 
    yield tf.cast(data['image'], dtype=tf.float32)

In [0]:
fis_score = frechet_inception_distance(ae, data_generator(), tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

calculating the inception images sigma ...


AttributeError: ignored

## perceptual_path_length_score

In [0]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [0]:
ppl_mean_score = perceptual_path_length_score(ae, data_generator(), tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

ValueError: ignored

## precision score

In [0]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [0]:
_precision_score = precision_score(ae, data_generator(), tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

ValueError: ignored

## recall score

In [0]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [0]:
_recall_score = recall_score(ae, data_generator(), tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

ValueError: ignored

# Image Generation

## image reconstruction

### Training dataset

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(recording_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, data_generator(), save_dir)

0it [00:00, ?it/s]


ValueError: ignored

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(recording_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, data_generator(), save_dir)

0it [00:00, ?it/s]


ValueError: ignored

## with Randomness

In [0]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(recording_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, data_generator(), save_dir)

InvalidArgumentError: ignored

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(recording_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, data_generator(), save_dir)

InvalidArgumentError: ignored

### Complete Randomness

In [0]:
from training.generators.image_generation_testing import generate_images_randomly

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(recording_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

0it [00:00, ?it/s]


ValueError: ignored

In [0]:
from training.generators.image_generation_testing import interpolate_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(recording_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, data_generator(), save_dir)

10it [00:00, 140.72it/s]
  0%|          | 0/15 [00:00<?, ?it/s]


InvalidArgumentError: ignored